In [1]:
import boto3
import datetime as dt
import pytz
import time
import uuid
import json
import s3fs
from urllib.parse import urlparse
import pandas as pd
import requests
import numpy as np
from decimal import *
from io import StringIO


In [2]:


cookies = {
}

data = '{"email":"btomasette@camelotsmm.com", "password":"Dunmore1"}'

auth = 'https://facebook.api.beeswax.com/rest/authenticate'

get_advertisers = 'https://facebook.api.beeswax.com/rest/advertiser'

save_report = 'https://facebook.api.beeswax.com/rest/report_save'

report_queue = 'https://facebook.api.beeswax.com/rest/report_queue'

list_item_bulk = 'https://facebook.api.beeswax.com/rest/list_item_bulk'

get_lists = 'https://facebook.api.beeswax.com/rest/custom_list'

r = requests.post(auth, cookies=cookies, data=data)

tz = pytz.timezone('EST')

today = dt.datetime.now(tz)
sevenday = today - dt.timedelta(days=7)


In [3]:

# query athena view of impression, click, and joined conversion data for tag_id 3
query = """
SELECT  conversions,
         campaign_id,
         impressions,
         clicks,
         app_bundle,
         ad_position,
         geo_region,
         geo_country,
         platform_browser,
         platform_os,
         rewarded,
         platform_carrier,
         platform_device_make,
         platform_device_model,
         video_player_size,
         video_completes,
         content_language,
         time_of_week,
         companion_views,
         companion_clicks,
         banner_width,
         banner_height,
         inventory_source,
         inventory_interstitial,
         spend
FROM "fb-prod"."fbbatchprediction" 
""" 


In [4]:

# set database to facebook
database = 'fb-prod'
#set s3 output file for athena query
s3_output = 's3://fb-beeswax/brian/athena/%s/'%(dt.datetime.now(tz).strftime('%Y-%m-%d-%H%M%S'))

#Function for starting athena query
def run_query(query, database, s3_output):
    client = boto3.client('athena', region_name='us-east-1')
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    print('Execution ID: ' + response['QueryExecutionId'])
    return response

#run athena query and kick back job id
job = run_query(query, database, s3_output)

job_id = job['QueryExecutionId']
client = boto3.client('athena', region_name='us-east-1')
res = client.get_query_execution(QueryExecutionId= job_id)
x = 0

# wait for athena to return results
while res['QueryExecution']['Status']['State'] != 'SUCCEEDED':
    print("it's been {} seconds".format(str(x)))
    time.sleep(10)
    x = x + 10
    res = client.get_query_execution(QueryExecutionId= job_id)

#set output location for query results
output = res['QueryExecution']['ResultConfiguration']['OutputLocation']


Execution ID: 8f52db4b-d080-4850-a929-a69b021fc7d1
it's been 0 seconds
it's been 10 seconds


In [5]:

#set aws machine learning client and build schema for datasource
mlclient = boto3.client('machinelearning', region_name='us-east-1')

data_id = str(uuid.uuid4())
data_name = 'fb-datasource-%s'%(dt.datetime.now(tz).strftime('%Y-%m-%d-%H%M%S'))

schema = {
        'excludedAttributeNames': [], 
        'version': '1.0', 
        'dataFormat': 'CSV', 
        'rowId': None, 
        'dataFileContainsHeader': True, 
        'attributes': [
            {
                'attributeName': 'conversions', 
                'attributeType': 'BINARY'
            }, {
                'attributeName' : 'campaign_id',
                'attributeType' : 'CATEGORICAL'
            }, {
                'attributeName': 'impressions', 
                'attributeType': 'NUMERIC'
            }, {
                'attributeName': 'clicks', 
                'attributeType': 'NUMERIC'
            }, {
                'attributeName': 'app_bundle', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'ad_position', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'geo_region', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'geo_country', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'platform_browser', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'platform_os', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'rewarded', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'platform_carrier', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'platform_device_make', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'platform_device_model', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'video_player_size', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'video_completes', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'content_language', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'time_of_week', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'companion_views', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'companion_clicks', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'banner_width', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'banner_height', 
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'inventory_source',
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'inventory_interstitial',
                'attributeType': 'CATEGORICAL'
            }, {
                'attributeName': 'spend',
                'attributeType': 'NUMERIC'
            }
        ]
    }


# load new datasource into aws machine learning
response = mlclient.create_data_source_from_s3(DataSourceId=data_id, DataSourceName=data_name, DataSpec={"DataLocationS3" : output, "DataSchema" : json.dumps(schema)}, ComputeStatistics=True)


In [ ]:

#build new batch prediction and set output file in S3
prediction_id = str(uuid.uuid4())
prediction_name = 'fb-prediction-%s'%(dt.datetime.now(tz).strftime('%Y-%m-%d-%H%M%S'))
prediction_output = 's3://fb-beeswax/brian/ml/%s/'%(dt.datetime.now(tz).strftime('%Y-%m-%d-%H%M%S'))
prediction_data_source_id = response['DataSourceId']
pred = mlclient.create_batch_prediction(BatchPredictionId=prediction_id, BatchPredictionName=prediction_name, MLModelId='fd8e2261-bc58-4e6a-a1af-5846c60d5df7', BatchPredictionDataSourceId=prediction_data_source_id , OutputUri=prediction_output)

get_pred = mlclient.get_batch_prediction(BatchPredictionId=pred['BatchPredictionId'])
gpt = 0
while get_pred['Status'] != 'COMPLETED':
    time.sleep(30)
    gpt = gpt + 30
    print("batch prediction processing for %s seconds" %gpt)
    get_pred = mlclient.get_batch_prediction(BatchPredictionId=pred['BatchPredictionId'])


batch prediction processing for 30 seconds
batch prediction processing for 60 seconds
batch prediction processing for 90 seconds
batch prediction processing for 120 seconds
batch prediction processing for 150 seconds


In [ ]:

#load original athena result into a dataframe
df = pd.read_csv(output)

#build score output url
o = urlparse(output)
score = get_pred['OutputUri'] + 'batch-prediction/result/' + get_pred['BatchPredictionId'] + '-' + o[2].split('/')[4].split('.')[0] + '.csv.gz'

#load score into a dataframe
ds = pd.read_csv(score, compression='gzip')

df['score'] = ds['score']


In [ ]:
#calculate range of scores
#range = df['score'].max() - df['score'].min()

#zero out any items below 0.009

#df['bid_mod'] = df['score'].apply(lambda y : 0 if y < 0.01 else y)

s3 = boto3.client(
    's3', region_name='us-east-1')
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)
res = s3.put_object(Body = csv_buffer.getvalue(),
                               ContentType='text/csv',
                               Bucket='fb-beeswax',
                               Key = 'brian/ml-prediction/{}/{}.csv'.format(dt.datetime.now(tz).strftime('%Y-%m-%d'), prediction_name ))
